In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import roc_auc_score, confusion_matrix
from catboost import CatBoostClassifier

### **1) Предобработка**

Анализ крови - это комплексный анализ, в ходе которого одномоментно оценивается сразу множество показателей (гемоглобин, эритроциты, лейкоциты, моноциты, etc). Для эффективной интерпретации результатов необходим анализ сразу всех признаков, получение каких-либо результатов (постановка диагноза) на основе только одного показателя представляется невозможным. К сожалению, в представленном датасете отсутствует ID анализа, что не позволяет понять, какие измерения относятся к одному анализу. 
Однако при внимательном анализе датасета можно выявить ряд факторов, позволяющих достаточно достоверным образом объединить измерения в один анализ. 
1) У ряда пациентов есть комментарии (записи в колонке 'Значение кол. показателя'). Все измерения у одного пациента, имеющие одинаковые комментарии относятся к одному анализу. Это представляется очевидным, так как вероятность того, что в разных анализах в комментариях написано одно и то же (в ряде случаев с грамматическими ошибками), пренебрежимо мала. Примеры комментариев: 
    - 'среди моноцитоидных  клеток **2,5** промоноцита.'
    - '**истиное** число лейкоцитов **20,6**'
    - '**тромболциты** проверены по мазку'
2) Если посмотреть на исходный датасет в оригинальном порядке строк, то можно отметить чередование определенных показателей, формирующих последовательности. Например, у пациента лейкоциты - гемоглобин - ... - снова лейкоциты - снова гемоглобин и т.д. Логично, что каждая такая последовательность - это отдельный анализ. Данный метод является менее точным, но в условиях имеющихся данных представляется достаточно адекватным.

Следуя описанной методологии произведено объединение различных измерений по анализам.

Вручную после этого были внесены некоторые изменения. Это ведет к нарушению воспроизводимости результатов, но с учетом качества предоставленных данных представляется позволительным. Исходя из логических умозаключений в ряде случаев ID были удалены. IDб созданные на основе комментариев и посследовательностей, были объединены в колонке 'Final Lab ID'

### **2) Сводная таблица**

Так как у нас мало пациентов с определенными диагнозами, и наша модель предназначена для скрининга, а не для точной диагностики, т.е. для выявления пациентов с лейкозами среди пациентов без лейкозов, то будем проводить бинарную классификацию между здоровыми пациентами и пациентами с лейкозами

### **3) Добавление здоровых пациентов из другого датасета**

### **4) Классфикация**

In [16]:
# Загрузим подготовленный датасет
all_patients = pd.read_csv('all_patients.csv')

Все признаки в двух сравниваемых группах имеют относительно схожие значения (нет такого, что в одной группе среднее значения признака = 1, а вдругой = 10), что говорит о правильности слияния данных. Однако между группами имеются значимые различия. Они могут быть обусловлены как истинными различиями между пациентами с лейкозами и без, так и различиями из-за того, что данные взяты из разных датасетов.

In [17]:
# Разделим на test и train так, чтобы 
# 1) была стратификация по диагнозам
# 2) строки для одной истории болезни были либо в test, либо в train

# Assuming 'Диагноз' is the target variable and 'ID' is the grouping variable
X = all_patients.drop(['Диагноз'], axis=1)
y = all_patients['Диагноз']
groups = all_patients['ID истории болезни']

# Initialize GroupShuffleSplit with test size and random state
gss = GroupShuffleSplit(n_splits=1, test_size=0.3, random_state=12)

# Perform the stratified split
for train_index, test_index in gss.split(X, y, groups=groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [18]:
# Удалим ненужные колонки
X_train = X_train.drop('ID истории болезни', axis=1)
X_test = X_test.drop('ID истории болезни', axis=1)

# Объединим, чтобы перемешать
train = pd.concat([X_train, y_train], axis=1, ignore_index=True)
test = pd.concat([X_test, y_test], axis=1, ignore_index=True)

# Перемешаем
train = train.sample(frac=1, random_state=42)
test = test.sample(frac=1, random_state=42)

# Разделим
X_train, y_train = train.iloc[:, :-1], train.iloc[:, -1]
X_test, y_test = test.iloc[:, :-1], test.iloc[:, -1]

In [19]:
# Обучим модель
model = CatBoostClassifier(loss_function='Logloss', iterations=100, learning_rate=0.1,
                           verbose=False, auto_class_weights='Balanced')
model.fit(X_train, y_train)

# Предскажем на тестовой выборке
y_pred = model.predict(X_test)
# Посчитаем вероятности
y_proba = model.predict_proba(X_test)[:, 1]

# Посчитаем чувствительность и специфичность
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

# Посчитаем ROC AUC
roc_auc = roc_auc_score(y_test, y_proba)

print("Sensitivity:", round(sensitivity, 5))
print("Specificity:", round(specificity, 5))
print("ROC AUC:    ", round(roc_auc, 5))

Sensitivity: 0.9108
Specificity: 0.99248
ROC AUC:     0.99512


1) В настоящем исследовании опробована методология объединения данных в анализы в предоставленном для работы датасете, а также добавление данных из другого датасета.
2) Получена модель для скрининга на лейкозы, отличающаяся высокой чувствительностью и очень высокой специфичностью.
3) Ключевое затруднение - это данные. К сожалению, в них отсутствуют анализы для здоровых пациентов, что вынудило добавить информацию из другого датасета. При этом не ясно, обусловлена ли эффективность работы модели наличием истинных различий между здоровыми пациентами и пациентами с лейкозами, либо же отличиями между двумя датасетами. 
4) Для создания более эффективных и транспаретных моделей машинного обучения необходимо обеспечение качественными данными, что в контексте поставленной для решения задачи означает предоставление информации не только о пациентах, страдающих заболеваниями, но и о здоровых пациентах.

In [20]:
# Сохранение
model.save_model("server/model.cbm")